In [1]:
import os
import matplotlib.pyplot as plt
from typing import Callable, Generator, Optional, Tuple

HOME = os.getcwd()
print(HOME)

/Users/owwl/Downloads/object_counting/track_and_count_people


In [8]:
SOURCE_VIDEO_PATH = os.path.join(HOME,"/Users/owwl/Downloads/object_counting/video5.mp4")
TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.replace(".mp4","_output.mp4")

In [9]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.5 🚀 Python-3.8.12 torch-2.4.1 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 335.1/460.4 GB disk)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 335.1/460.4 GB disk)


In [10]:
import supervision as sv
import numpy as np
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.18.0


In [11]:
from ultralytics import YOLO
MODEL = "/Users/owwl/Downloads/object_counting/best5.pt"
model = YOLO(MODEL)
model.fuse()

Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


In [12]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - person
selected_classes = [0]

# Frame inference to configure the line

In [13]:
LINE_START = sv.Point(0,  600)
LINE_END = sv.Point(1920, 600)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=1)

byte_tracker = sv.ByteTrack(track_thresh=0.05, track_buffer=30, match_thresh=0.8, frame_rate=24)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
box_annotator = sv.BoxAnnotator(thickness=3, text_thickness=0, text_scale=.5)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# acquire first video frame
iterator = iter(generator)
frame = next(iterator)

# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False,device='0',conf=.015, iou=.02,imgsz=1280)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]
# tracking detections
detections = byte_tracker.update_with_detections(detections)
line_zone.trigger(detections)

labels = [
    f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
    for _,_,confidence,class_id,tracker_id,_
    in detections
]

# annotate and display frame
annotated_frame = trace_annotator.annotate(scene=frame.copy(),detections=detections)
anotated_frame=box_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
anotated_frame = line_zone_annotator.annotate(anotated_frame, line_counter=line_zone)

plt.figure(figsize=(16, 16))
plt.imshow(anotated_frame[..., ::-1])
plt.axis('on')
plt.show()

Ultralytics YOLOv8.1.5 🚀 Python-3.8.12 torch-2.4.1 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


# Video Inference

In [ ]:
byte_tracker = sv.ByteTrack(track_thresh=0.05, track_buffer=30, match_thresh=0.8, frame_rate=24)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)
box_annotator = sv.BoxAnnotator(thickness=3, text_thickness=0, text_scale=.5)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=1)

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False,device='0',conf=.015, iou=.02,imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    # update line counter
    line_zone.trigger(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for _,_,confidence,class_id,tracker_id,_
        in detections
    ]
    # annotate and display frame
    annotated_frame = trace_annotator.annotate(scene=frame.copy(),detections=detections)
    anotated_frame=box_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    anotated_frame = line_zone_annotator.annotate(anotated_frame, line_counter=line_zone)
    return anotated_frame


def process_video(
    source_path: str,
    target_path: str,
    callback: Callable[[np.ndarray, int], np.ndarray],
    debug: bool,
) -> None:
    source_video_info = sv.VideoInfo.from_video_path(video_path=source_path)
    with sv.VideoSink(target_path=target_path, video_info=source_video_info) as sink:
        for index, frame in enumerate(
            sv.get_video_frames_generator(source_path=source_path)
        ):
            result_frame = callback(frame, index)
            sink.write_frame(frame=result_frame)
 
            if(debug): 
                plt.figure(figsize=(16, 16))
                plt.imshow(result_frame[..., ::-1])
                plt.axis('on')
                plt.show()
                if(index == 0): break

process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback,
    debug=False
)